In [ ]:
import os
import networkx as nx
import community
import matplotlib.pyplot as plt
from community import community_louvain
import numpy as np
import pandas as pd
import re
from webweb import Web
import operator
import math
import itertools
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
from pandas import ExcelWriter
from pandas import ExcelFile
from collections import Counter
from collections import defaultdict
from networkx import edge_betweenness_centrality as betweenness
import networkx as nx
import community
import matplotlib.pyplot as plt
from community import community_louvain

In [ ]:
#Directed, Weighted
def exp_directed_weighted(df_exp,year):
    df_exp = df_exp[df_exp['Year']==year]
    nodes = np.unique(df_exp[['ReporterName','PartnerName']]).tolist()
    edges = df_exp[['ReporterName','PartnerName','Value']].values.tolist()
    
    for edge in edges:
        edge[2] = edge[2]/max(map(lambda x: x[2], edges))

    labels = {}
    for i in range(len(nodes)):
        labels[nodes[i]] = nodes[i]
        

    return nodes,labels,edges


In [ ]:
#Directed,weighted graph
def graph_d_w(nodes,weighted_edges):
    g = nx.DiGraph()
    for i in weighted_edges:
        g.add_edge(i[0], i[1], weight=i[2]) 
    return g

In [ ]:
def calculate_purity(df_exp_prod):
    purity_scores_list = []

    nodes_prev,labels_prev,edges_prev = exp_directed_weighted(df_exp_prod,1996)
    G = nx.Graph()
    G.add_nodes_from(nodes_prev)
    for i in edges_prev:
        G.add_edge(i[0], i[1], weight=i[2]) 
    partition = community_louvain.best_partition(G,weight='weight')
    prev_communities = defaultdict(list)
    for key, value in sorted(partition.items()):
        prev_communities[value].append(key)


    for year in range(1997, 2019,1):
        nodes,labels,edges = exp_directed_weighted(df_exp_prod,year)
        G = nx.Graph()
        G.add_nodes_from(nodes)
        for i in edges:
            G.add_edge(i[0], i[1], weight=i[2]) 
        partition = community_louvain.best_partition(G,weight='weight')
        communities = defaultdict(list)
        for key, value in sorted(partition.items()):
            communities[value].append(key)

        purity = 0
        country_purity = {}
        comm1 = set()
        comm2 = set()
        for country in nodes_prev:
            for c in prev_communities.values():
                if country in c:
    #                 print(c)
                    comm1 = set(c)
            for c in communities.values():
    #             print(c)
                if country in c:
                    comm2 = set(c)
            inter = comm1.intersection(comm2)
            uni = comm1.union(comm2)
            purity  = (len(inter)/len(uni))
            country_purity[country] = purity


        total_purity_score =0
        for comm in prev_communities.values():
            purity_score_comm = 0
            for country in comm:
                purity_score_comm += country_purity[country]
            purity_score_comm/= len(comm)

            total_purity_score+=purity_score_comm


        purity_scores_list.append(total_purity_score/len(prev_communities))

        nodes_prev,labels_prev,edges_prev = nodes, labels, edges
        prev_communities = communities
        
    return purity_scores_list


In [ ]:
def draw_plot(df,year):
    nodes,labels,edges = exp_directed_weighted(df,year)
    # print(len(edges_fuel))
    G = graph_d_w(nodes,edges)
    d = nx.degree(G)  
    partition = community_louvain.best_partition(G,weight='weight')

    size = float(len(set(partition.values())))
    pos = nx.spring_layout(G,iterations=10)
    plt.figure(figsize=(10, 10))  # image is 8 x 8 inches
    plt.axis('off')

    nx.draw_networkx_nodes(G, pos, node_size=[v[1] for v in d], cmap=plt.get_cmap('jet'), node_color=list(partition.values()))
    nx.draw_networkx_edges(G, pos, alpha=0.01)
    plt.savefig('community.png')

    return partition


In [ ]:
#DataFrame Fuel
df_exp_fuel = pd.read_csv('2000FuelExports.csv')
df_exp_fuel['ReporterName'] = df_exp_fuel['ReporterName'].astype('str')
df_exp_fuel['PartnerName'] = df_exp_fuel['PartnerName'].astype('str')

df_exp_fuel['ReporterName'] = df_exp_fuel['ReporterName'].str.strip()
df_exp_fuel['PartnerName'] = df_exp_fuel['PartnerName'].str.strip()
years = np.unique(df_exp_fuel['Year']).tolist()


In [ ]:
#DataFrame Cement
df_exp_cement = pd.read_csv('CementExports.csv')
df_exp_cement['ReporterName'] = df_exp_cement['ReporterName'].astype('str')
df_exp_cement['PartnerName'] = df_exp_cement['PartnerName'].astype('str')

df_exp_cement['ReporterName'] = df_exp_cement['ReporterName'].str.strip()
df_exp_cement['PartnerName'] = df_exp_cement['PartnerName'].str.strip()

In [ ]:
#DataFrame AutoMobiles
df_exp_auto = pd.read_csv('EnginesExports.csv')
df_exp_auto['ReporterName'] = df_exp_auto['ReporterName'].astype('str')
df_exp_auto['PartnerName'] = df_exp_auto['PartnerName'].astype('str')

df_exp_auto['ReporterName'] = df_exp_auto['ReporterName'].str.strip()
df_exp_auto['PartnerName'] = df_exp_auto['PartnerName'].str.strip()

In [ ]:
#partitions- Communities
partition_fuel = draw_graph(df_exp_fuel,2005)
partition_cement = draw_graph(df_exp_cement,2005)
partition_auto = draw_graph(df_exp_auto,2005)

In [ ]:
v = defaultdict(list)
for key, value in sorted(partition.items()):
    v[value].append(key)
print(v)

In [ ]:
#Community detection through betweenes centrality
def most_central_edge(G):
    centrality = nx.edge_betweenness_centrality(G)
    new_centrality = {}
    for edge,cen in centrality.items():
        new_centrality[edge] = G[edge[0]][edge[1]]['weight']*G.degree[edge[0]]*G.degree[edge[1]]*cen
    return max(new_centrality, key=new_centrality.get)

G = nx.Graph()
nodes,labels,edges = exp_directed_weighted(df_exp_fuel,2017)
# print(nodes)
G.add_nodes_from(nodes)
for edge in edges:
    G.add_edge(edge[0],edge[1],weight=edge[2])
comp = nx.algorithms.community.centrality.girvan_newman(G, most_valuable_edge=most_central_edge)


limited = itertools.takewhile(lambda c: len(c) <= 10, comp)

for communities in limited:
    print(tuple(sorted(c) for c in communities)) 



In [ ]:
#purity scores
purity_score_fuel = calculate_purity(df_exp_fuel)
purity_score_cement = calculate_purity(df_exp_cement)
purity_score_auto = calculate_purity(df_exp_auto)


In [ ]:
#community stability 
plt.xlabel("year")
plt.ylabel("Stability of Communities in consecutive years")
plt.plot(years[:len(years)-1],purity_score_fuel,'-',label='fuel')
plt.plot(years[:len(years)-1],purity_score_cement,'-',label='cement')
plt.plot(years[:len(years)-1],purity_score_auto,'-',label='AutoMobiles')
plt.legend()
plt.savefig('community_stability.png')